# Reformater le vocabulaire des solutions
- en fonction du nombres de lignes à mettre sur la page qui commence la catégorie

In [1]:
# -*- coding: utf8 -*-
from os.path import expanduser
import time
import pyperclip
home = expanduser("~")
annee="18"
nKalaba="K4"
repertoire=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/"+annee+"-%s"%nKalaba
serie=repertoire+"/"
nomVocabulaire=serie+"Vocabulaire-Corr.tex"
debug=False

In [2]:
#########################IMPORTS############################################
import codecs, optparse
import re, random
import sys,os,time
import string
import yaml, warnings

In [3]:
with codecs.open(nomVocabulaire, 'r',encoding="utf8") as input:
    linesVocabulaire="".join(input.readlines()).split("\n")

In [4]:
linesSortie=[]
noms=[]
adjectifs=[]
verbes=[]
determinants=[]
prepositions=[]
contenant=[]
categories=[noms,adjectifs,verbes,determinants,prepositions]
nomCategories=u"Noms,Adjectifs,Verbes,Déterminants,Prépositions".split(u",")
destListe=contenant
for e in linesVocabulaire:
    if e.startswith(u"\\item NOMS"):
        destListe=noms
    elif e.startswith(u"\\item ADJECTIFS"):
        destListe=adjectifs
    elif e.startswith(u"\\item VERBES"):
        destListe=verbes
    elif e.startswith(u"\\item PRÉPOSITIONS"):
        destListe=prepositions
    elif e.startswith(u"\\item DÉTERMINANTS"):
        destListe=determinants
    else:
        destListe.append(e)

# Récupération du vocabulaire des catégories
- entête
- éléments

In [5]:
footers=[
    ur"\hline\end{tabular}\\%",
    ur"\vfill\null\columnbreak%"
]
def getHeader(locCat,lenHeader=5):
    result=None,None
    m=re.match(ur"^.*multicols..(\d+).$",locCat[0])
    if m:
        multicols[n]=m.group(1)
        result=locCat[1:lenHeader],locCat[lenHeader:]
    return result
    
multicols={}
headers={}
for n,cat in enumerate(categories):
    headers[n],tempCat=getHeader(cat)
    categories[n]=[]
    for ligne in tempCat:
        m=re.match(ur"^([^&]+) & ([^&]+) & ([^&]+) \\",ligne)
        if m and not ligne.startswith("Mot & Roman & Glose"):
            categories[n].append(ligne)        

# Restructuration du Vocabulaire
- par catégorie
 - changement éventuel du nombre de colonnes
 - lignes sur la première page
 
Découper en tableaux

In [12]:
nbLignesPage=32 #32 paysage ou 48 portrait

def makeTableaux(lignes,cols,longueur):
    resTableaux=[]
    lenLignes=len(lignes)
    if lenLignes>0:
        tailleTableaux=cols*longueur
        if tailleTableaux<=lenLignes+(cols-1):
            if longueur>1 or tailleTableaux<=lenLignes :
                for c in range(cols):
                    resTableaux.extend(headers[n])
                    resTableaux.extend(lignes[c*longueur:(c+1)*longueur])
                    resTableaux.extend(footers)
            else:
                for c in range(lenLignes):
                    resTableaux.extend(headers[n])
                    resTableaux.extend(lignes[c*longueur:(c+1)*longueur])
                    resTableaux.extend(footers)
            return resTableaux,lignes[tailleTableaux:]
        else:
            l=int(float(lenLignes)/cols)
            if l*cols<lenLignes:
                l+=1
            print "makeTableaux",l,cols,lenLignes
            return makeTableaux(lignes,cols,l)
    else:
        return None,None

sortieLignes=[
    ur"%% script : Kalaba-ReformatageVocabulaire-Corr",
    ur"%% run : %s"% time.strftime("%y%m%d-%H%M"),
    ur"\begin{itemize}"
    ]
rawinput=False
# catCols=[3,3,2,4,4]      #K1
# catPrems=[29,1,13,4,10]  #K1
# catCols=[3,3,2,3,4]      #K3
# catPrems=[29,6,17,21,10]  #K3
catCols=[3,3,2,3,4]      #K4
catPrems=[29,6,10,19,10]  #K4
for n,cat in enumerate(categories):
    sortieLignes.append(ur"\Needspace{8\baselineskip}%")
    sortieLignes.append(ur"\item %s\\[-3ex]"%nomCategories[n].upper())
    question=u"%s colonnes pour la catégorie %s\n"%(catCols[n],nomCategories[n])
    if rawinput:
        strNbColonnes=raw_input(question.encode("utf8"))
        if strNbColonnes:
            nbColonnes=int(strNbColonnes)
        else:
            nbColonnes=catCols[n]
        strNbLignes=raw_input(u"combien de lignes maximum pour le premier tableau : %s ?\n"%catPrems[n])
        if strNbLignes:
            nbLignes=min(int(strNbLignes),nbLignesPage)
        else:
            nbLignes=catPrems[n]
    else:
        nbColonnes=catCols[n]
        nbLignes=catPrems[n]
    sortieLignes.append(ur"\begin{multicols}{%d}"%nbColonnes)
    nbElements=len(cat)
    nbPremiersTableaux=nbColonnes*nbLignes
    print nbElements,nbPremiersTableaux
    tableau,restes=makeTableaux(cat,nbColonnes,nbLignes)
    print u"\n".join(tableau)
    sortieLignes.extend(tableau)
    while restes:
        tableau,restes=makeTableaux(restes,nbColonnes,nbLignesPage)
        #print u"\n".join(tableau)
        sortieLignes.extend(tableau)
    print "pop",sortieLignes.pop()
    sortieLignes.append(ur"\end{multicols}")
    if restes: sortieLignes.append(ur"\Needspace{8\baselineskip}%")
sortieLignes.append(ur"\end{itemize}")

146 87
\begin{tabular}[t]{|l|l|l|}
\addlinespace[-1.0em]\hline
Mot & Roman & Glose  \\
\hline\strutgh{14pt}%
\KalebASgAbs & \KalebASgAbsP & \blanc{\KalebASgAbsG} \\
\KalebASgDat & \KalebASgDatP & \blanc{\KalebASgDatG} \\
\KalebASgErg & \KalebASgErgP & \blanc{\KalebASgErgG} \\
\KalebASgObl & \KalebASgOblP & \blanc{\KalebASgOblG} \\
\MahiraCSgAbs & \MahiraCSgAbsP & \blanc{\MahiraCSgAbsG} \\
\MahiraCSgDat & \MahiraCSgDatP & \blanc{\MahiraCSgDatG} \\
\MahiraCSgErg & \MahiraCSgErgP & \blanc{\MahiraCSgErgG} \\
\MahiraCSgObl & \MahiraCSgOblP & \blanc{\MahiraCSgOblG} \\
\VioletteDSgAbs & \VioletteDSgAbsP & \blanc{\VioletteDSgAbsG} \\
\VioletteDSgErg & \VioletteDSgErgP & \blanc{\VioletteDSgErgG} \\
\arbreBDuAbs & \arbreBDuAbsP & \blanc{\arbreBDuAbsG} \\
\arbreBPlAbs & \arbreBPlAbsP & \blanc{\arbreBPlAbsG} \\
\arbreBPlObl & \arbreBPlOblP & \blanc{\arbreBPlOblG} \\
\arbreBSgAbs & \arbreBSgAbsP & \blanc{\arbreBSgAbsG} \\
\arbreBSgObl & \arbreBSgOblP & \blanc{\arbreBSgOblG} \\
\blessureAPlErg & \bl

In [13]:
pyperclip.copy("\n".join(sortieLignes))